In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv

In [2]:
csv_file = open("graphics_cards.csv", "w", newline="")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(
    ["ItemID", 
    "Title", 
    "Branding", 
    "Rating", 
    "Number of Ratings", 
    "Price", 
    "Shipping", 
    "Image URL", 
    "Max Resolution", 
    "DisplayPort", 
     "HDMI", 
     "DirectX", 
     "Model"])


119

In [11]:
import time
import random
csv_writer = csv.writer(csv_file)
page_limit = 9
sleep_duration = 300  # Thời gian sleep (số giây)
headers_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.37",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.37",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.37",
    # Thêm các User-Agent khác tùy ý vào danh sách này
]
for page in range(1, 101):
    headers = {
        "User-Agent": random.choice(headers_list)  # Chọn ngẫu nhiên một User-Agent từ danh sách
    }
    url = f"https://www.newegg.com/GPUs-Video-Graphics-Cards/SubCategory/ID-48/Page-{page}?Tid=7709"
    print("wakeup")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    #lấy mỗi div item
    products = soup.find_all("div", class_="item-cell")
    if products == []:
        print(f"Page {page} is null.")
    for product in products:
        item_id = product.find("div", class_="item-container").get("id")
        title = product.find("a", class_="item-title").text.strip()
        branding_element = product.find("a", class_="item-brand")
        if branding_element:
            branding = branding_element.img["title"]
        else:
            branding = None
        
        rating_element = product.find("a", class_="item-rating")
        if rating_element:
            rating = rating_element.get("title")
            rating = re.search(r"(\d+(\.\d+)?)", rating).group(1) if rating else None
        else:
            rating = None
        
        num_ratings_element = product.find("span", class_="item-rating-num")
        if num_ratings_element:
            num_ratings = re.search(r"(\d+)", num_ratings_element.text).group(1) if num_ratings_element.text else None
        else:
            num_ratings = None

        price_element = product.find("li", class_="price-current").text.strip()
        price_match = re.search(r"\$(\d{1,3}(,\d{3})*(\.\d+)?)", price_element)
        if price_match is not None:
            if price_match:
                price = float(price_match.group(1).replace(',', ''))
            else:
                price = None
        else:
            price = None
        shipping_element = product.find("li", class_="price-ship")
        if shipping_element:
            shipping = shipping_element.text.strip()
        else:
            shipping = None
            
        image_url = product.find("img")["src"]
        
        # Truy cập vào trang chi tiết sản phẩm để lấy thông tin chi tiết
        details_url = product.find("a", class_="item-img")["href"]
        details_response = requests.get(details_url)
        details_soup = BeautifulSoup(details_response.text, "html.parser")
        
        
        # Tìm tất cả các bảng có class "table-horizontal"
        tables = details_soup.find_all("table", class_="table-horizontal")

        # Tạo một từ điển để lưu trữ thông tin
        details = {}

        # Lặp qua từng bảng để lấy thông tin
        for table in tables:
            caption = table.caption.get_text(strip=True)
            if caption in ["Model", "3D API", "Ports", "Details"]:
                rows = table.tbody.find_all("tr")
                for row in rows:
                    th = row.th.get_text(strip=True)
                    td = row.td.get_text(strip=True)
                    if th in ["Max Resolution", "DisplayPort", "HDMI", "DirectX", "Model"]:
                        details[th] = td

        max_resolution = details.get("Max Resolution", "")
        display_port = details.get("DisplayPort", "")
        hdmi = details.get("HDMI", "")
        directx = details.get("DirectX", "")
        model = details.get("Model", "")
        csv_writer.writerow([item_id, title, branding, rating, num_ratings, price, shipping, image_url, max_resolution, display_port, hdmi, directx, model])
    print(f"Page {page} done.")
    if page % page_limit == 0: # Kiểm tra nếu là trang đầu tiên trong mỗi lần request
        print("sleep")
        time.sleep(sleep_duration)  # Sleep trong 5 phút
print("csv_writer done")


wakeup
Page 82 done.
wakeup
Page 83 done.
wakeup
Page 84 done.
wakeup
Page 85 done.
wakeup
Page 86 done.
wakeup
Page 87 done.
wakeup
Page 88 done.
wakeup
Page 89 done.
wakeup
Page 90 done.
sleep
wakeup
Page 91 done.
wakeup
Page 92 done.
wakeup
Page 93 done.
wakeup
Page 94 done.
wakeup
Page 95 done.
wakeup
Page 96 done.
wakeup
Page 97 done.
wakeup
Page 98 done.
wakeup
Page 99 done.
sleep
wakeup
Page 100 done.
csv_writer done


In [12]:
csv_file.close()